In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [7]:
md1 = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [8]:
pm = PETsARD.HyperProcessor(metadata)

In [12]:
pm.get_config()

{'missingist': {'gen': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152362740>,
  'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x122228940>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1523626e0>,
  'edu': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152363340>,
  'mar': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152362bc0>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152363250>,
  'dep': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152362530>,
  'pir': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152362800>,
  'gh': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152362320>,
  'mets': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152362500>,
  'qm': <PETsARD.Processor.Missingist.Missingist_Drop at 0x152362980>,
  'dia': <PETsARD.Processor.Missingist.Missingist_Mean at 0x152363550>},
 'outlierist': {'gen': None,
  'age': <PETsARD.Processor.Outlierist.Outlierist_IQR at 0x110abfee0>,
  'race': None,
  'ed

In [9]:
from PETsARD.Processor.Missingist import Missingist_Mean

In [13]:
config_1 = {'missingist': {'gen': Missingist_Mean(),
  'age': Missingist_Mean()},
 'outlierist': {'gen': None,
  'age': None},
 'encoder': {'gen': None,
  'age': None},
 'scaler': {'gen': None,
  'age': None}}

In [ ]:
pm.update_config(config_1)

In [ ]:
pm._sequence

['missingist']